# Dados UFRN - Análise


## Descrição do projeto

## Descrição dos dados


## Autores


## Vídeo explicativo



Importando bibliotecas a serem usadas

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#conda install -c conda-forge seaborn

Abertura de dados

In [32]:
#Carregando arquivo referente as unidades
data_unidades = pd.read_csv("Dados/unidades/unidades.csv", sep=';',
                            usecols=["id_unidade", "nome_unidade", "sigla", "municipio","id_unidade_gestora",
                                     "id_unidade_responsavel", "unidade_responsavel", "classificacao_unidade"])


#Selecionando somente os institutos e os centros
centro_ou_inst = [nome.startswith("CENTRO") or nome.startswith("INSTITUTO") for nome in data_unidades.nome_unidade]
data_unidades_ci = data_unidades[centro_ou_inst]

#Carregando arquivo referente aos gastos
data_gastos = pd.read_csv("Dados/unidades/gastos-por-unidade.csv", 
                          sep=';',
                          usecols=["id_unidade", "natureza_despesa", "valor"])

#Convertendo coluna de String para float
data_gastos.valor = data_gastos.valor.replace('[R | ' ' | \$,.]', '', regex=True).astype(float) / 100

#Unindo as tabelas
dataM_unit_gas = pd.merge(data_unidades_ci, data_gastos, on="id_unidade")

#Criando pivot table para somar gasto por unidade
ptable = pd.pivot_table(dataM_unit_gas, index=["id_unidade", "nome_unidade"], values=["valor"], aggfunc=np.sum)

#Salvando essa pivot table num DataFrame
data_gastos_ok = pd.DataFrame(ptable.to_records())

link_unidades = data_unidades[["id_unidade", "id_unidade_gestora"]]

,id_unidade,id_unidade_gestora
0,8190,441
1,8772,605
2,176,440
3,119,439
4,389,441
5,5966,5965
6,61,1482
7,62,1482
8,10552,5965
9,2877,443


In [3]:
#Carregando arquivo dos docentes (link entre turmas e unidades gestoras)
link_docentes = pd.read_csv("Dados/docentes/docentes.csv",
                           sep=';',
                           usecols=["id_servidor", "id_unidade_lotacao"])

In [33]:
#Carregando arquivos de semestre e linkando

for i in range(2016,2018):
    for j in range(1,3):
        #Carregando strings com o caminho dos dados a serem carregados
        file_matricula = "Dados/matriculas/" + (str(i) + str(j)) + ".csv"
        file_turmas = "Dados/turmas/" + (str(i) + str(j)) + ".csv"
        
        #Carregando arquivo de matriculas de certo semestre para DataFrame
        data_matriculas = pd.read_csv(file_matricula, sep=';',
                                      usecols=["id_turma", "discente", "descricao"])
        #id_turma: necessario para linkar com "turmas"
        #discente: necessario para contagem
        #descricao: o que vamos contar
        
        #Separando valores de descricao irrelevantes para o problema
        tags_indesejadas = ["INDEFERIDO", "AGUARDANDO DEFERIMENTO", "EM ESPERA", 
                            "CUMPRIU", "TRANSFERIDO", "DISPENSADO", "EXCLUIDA"]
        
        #Selecionando apenas os valores nao irrelevantes (relevantes)
        data_matriculas = data_matriculas[~data_matriculas.descricao.isin(tags_indesejadas)]
        
        #Criando mapa para classificar APROVADO ou APROVADO POR NOTA como feedback positivo
        #E classificar todas outras tags como feedback negativo
        map_dict = {}
        for desc in data_matriculas.descricao:
            if desc == "APROVADO POR NOTA" or desc == "APROVADO":
                map_dict[desc] = 1
            else:
                map_dict[desc] = 0
            
        #Atualizar valores das tags com valores mapeados
        data_matriculas.descricao = data_matriculas.descricao.map(map_dict)
        
        #Carregando arquivo de turmas de certo semestre para DataFrame
        data_turmas = pd.read_csv(file_turmas, sep=';',
                                  usecols=["id_turma", "id_docente_interno", "nivel_ensino",
                                           "ano", "periodo","situacao_turma"])
        #id_turma: necessario para linkar com "matriculas"
        #id_docente_interno: necessario para linkar com "docentes"
        #nivel_ensino: restringir para GRADUACAO
        #ano: restringir para ano analisado
        #periodo: restringir para periodo analisado
        #situacao_turma: restringir para turmas consolidadas
        
        #Restringindo a busca para o ano procurado
        data_turmas = data_turmas[data_turmas.ano == i]
        
        #Restringindo a busca para o periodo procurado
        data_turmas = data_turmas[data_turmas.periodo == j]
        
        #Restringindo a busca para o curso superior
        data_turmas = data_turmas[data_turmas.nivel_ensino == "GRADUAÇÃO"]
        
        #Restringindo a busca para turmas consolidadas
        data_turmas = data_turmas[data_turmas.situacao_turma == "CONSOLIDADA"]
        
        #Dando merge nas tabelas turmas e matriculas
        dataM_mat_tur = pd.merge(data_turmas, data_matriculas, on="id_turma")
        
        #Criando pivot table para contar reprovacoes, aprovacoes...
        ptable = pd.pivot_table(dataM_mat_tur, index=["id_turma", "id_docente_interno", "descricao"],
                                values="ano", aggfunc="count").rename(columns={'ano': 'Contagem'})
        
        #Salvando essa pivot table num DataFrame
        data_cont_situ = pd.DataFrame(ptable.to_records())        
        
        #Corrigindo nome de coluna, para possibilitar merge
        data_cont_situ.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)
        
        #Executando merge
        dataM_mt_doc = pd.merge(data_cont_situ, link_docentes, on="id_servidor")
        
        #Criando pivot table para agregar contagens por unidade e entao por turma
        ptable = pd.pivot_table(dataM_mt_doc, index=["id_unidade_lotacao", "id_turma", "descricao"],
                               values="Contagem")
        
        #Salvando essa pivot table num DataFrame
        data_cont_unit = pd.DataFrame(ptable.to_records())
        
        #Adicionando nova coluna, referente a porcentagem de valores em "descricao" por "id_turma"
        data_cont_unit["% da turma"] = (data_cont_unit.Contagem / 
                                        data_cont_unit.groupby(["id_unidade_lotacao","id_turma"])
                                        .Contagem.transform(sum) * 100)
        
        ptable = pd.pivot_table(data_cont_unit, index=["id_unidade_lotacao", "id_turma" ,"descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        #Salvando essa pivot table num DataFrame
        data_cont_unit = pd.DataFrame(ptable.to_records())
        
        #Adicionando valores 0% nas turmas com 100% de aprovacao (ou reprovacao) para poder calcular a media
        for (unidade, turma), _count in data_cont_unit.groupby(["id_unidade_lotacao", "id_turma"])["descricao"].count().iteritems():
            if _count == 1:
                soh_aprovado_ou_nao = data_cont_unit[(data_cont_unit.id_turma == turma) & 
                                  (data_cont_unit.id_unidade_lotacao == unidade)]["descricao"]
   
                temp_df = pd.DataFrame(data={"id_unidade_lotacao": unidade,
                                             "id_turma": turma,
                                             "descricao": 1 - soh_aprovado_ou_nao,
                                             "% da turma": 0.0}, columns=data_cont_unit.columns)
                
                data_cont_unit = data_cont_unit.append(temp_df, ignore_index=True)
        
        #Calculando a media de aprovacao por unidade
        ptable = pd.pivot_table(data_cont_unit, index=["id_unidade_lotacao" ,"descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        #Salvando essa pivot table num DataFrame
        data_aprov_unit = pd.DataFrame(ptable.to_records()).rename(columns={"id_unidade_lotacao": "id_unidade"})
        print(data_aprov_unit)
   
        data_aprov_unit = pd.merge(link_unidades, data_aprov_unit, on="id_unidade")
        
        print(data_aprov_unit)
             
            
            
        

     id_unidade  descricao  % da turma
0            47          0   31.404177
1            47          1   68.595823
2            50          0   19.067450
3            50          1   80.932550
4            52          0   15.642860
5            52          1   84.357140
6            54          0    8.871416
7            54          1   91.128584
8            56          0   20.774895
9            56          1   79.225105
10           57          0   10.677905
11           57          1   89.322095
12           63          0    9.923341
13           63          1   90.076659
14           68          0   20.689655
15           68          1   79.310345
16           79          0    5.733851
17           79          1   94.266149
18           84          0   11.600448
19           84          1   88.399552
20          112          0   24.850639
21          112          1   75.149361
22          115          0   12.993554
23          115          1   87.006446
24          117          

In [98]:
sns.set()
data_gastos
# The four preset contexts, in order of relative size, are paper, notebook, talk, and poster
sns.set_context("notebook")
data_gastos.valor = data_gastos.valor.astype(float)
# plot a univariate distribution of observations.
#sns.kdeplot(int(data_gastos["valor"]))

ValueError: could not convert string to float: 'R$           389,20'